In [2]:
import json
from commons.config import MODANET_TRAIN_PATH, MODANET_TEST_PATH

In [5]:
loaded_json = None
with open(MODANET_TRAIN_PATH, 'r') as f:
    json_data = f.read().replace('\n', '')
    loaded_json = json.loads(json_data)

In [16]:
for k, _ in loaded_json.items():
    print(k)

year
images
info
categories
licenses
type
annotations


In [21]:
loaded_json['images'][0]

{'file_name': '0736791.jpg',
 'height': 600,
 'id': 736791,
 'license': 3,
 'width': 400}

In [22]:
len(loaded_json['images'])

52377

In [25]:
loaded_json['categories']

[{'id': 1, 'name': 'bag', 'supercategory': 'fashion'},
 {'id': 2, 'name': 'belt', 'supercategory': 'fashion'},
 {'id': 3, 'name': 'boots', 'supercategory': 'fashion'},
 {'id': 4, 'name': 'footwear', 'supercategory': 'fashion'},
 {'id': 5, 'name': 'outer', 'supercategory': 'fashion'},
 {'id': 6, 'name': 'dress', 'supercategory': 'fashion'},
 {'id': 7, 'name': 'sunglasses', 'supercategory': 'fashion'},
 {'id': 8, 'name': 'pants', 'supercategory': 'fashion'},
 {'id': 9, 'name': 'top', 'supercategory': 'fashion'},
 {'id': 10, 'name': 'shorts', 'supercategory': 'fashion'},
 {'id': 11, 'name': 'skirt', 'supercategory': 'fashion'},
 {'id': 12, 'name': 'headwear', 'supercategory': 'fashion'},
 {'id': 13, 'name': 'scarf/tie', 'supercategory': 'fashion'}]

In [26]:
len(loaded_json['annotations'])

265705

In [23]:
loaded_json['annotations'][0]

{'area': 1746,
 'bbox': [160, 247, 97, 18],
 'category_id': 2,
 'id': 0,
 'image_id': 736791,
 'iscrowd': 0,
 'segmentation': [[161,
   248,
   170,
   248,
   173,
   249,
   189,
   251,
   199,
   252,
   199,
   250,
   212,
   250,
   212,
   252,
   224,
   253,
   239,
   251,
   248,
   250,
   257,
   247,
   257,
   253,
   249,
   255,
   248,
   260,
   248,
   265,
   228,
   264,
   215,
   264,
   200,
   264,
   196,
   261,
   189,
   260,
   182,
   259,
   171,
   259,
   164,
   255,
   160,
   254,
   161,
   254]]}